# Tema 1 IA - Class Scheduler
### Alexandru LICURICEANU - 332CD

In [ ]:
# Prerequisites
import utils
import time
from copy import deepcopy
from heapq import heappop, heappush

from utils import ZILE as DAYS
from utils import INTERVALE as INTERVALS
from utils import SALI as CLASSROOMS
from utils import MATERII as SUBJECTS
from utils import PROFESORI as TEACHERS

MAX_INTERVALS = 7

CONSTRAINTS = 'Constrangeri'
PREFERRED = 'Preferred'
NOT_PREFERRED = 'Not_preferred'
CAPACITY = 'Capacitate'
PREFERRED_DAYS = 'Preferred_days'
NOT_PREFERRED_DAYS = 'Not_preferred_days'
PREFERRED_INTERVALS = 'Preferred_intervals'
NOT_PREFERRED_INTERVALS = 'Not_preferred_intervals'

input_files = {'dummy': 0, 'orar_mic_exact': 0, 'orar_mediu_relaxat': 0, 'orar_mare_relaxat': 0, 'orar_constrans_incalcat': 9}#, 'orar_bonus_exact': 0}

`parse_input_file()` Transforms the teachers' preferences into the formats used by the algorithms.

In [ ]:
def parse_input_file(input_file):
    data = utils.read_yaml_file(input_file)

    # Process the soft constraints for each teacher.
    for teacher in data[TEACHERS]:
        preferred = [constraint for constraint in data[TEACHERS][teacher][CONSTRAINTS] if not constraint.startswith('!')]
        not_preferred = [constraint.strip('!') for constraint in data[TEACHERS][teacher][CONSTRAINTS] if constraint.startswith('!')]

        # Convert intervals from '8-12' to (8, 12), etc.
        preferred_intervals = [tuple(map(int, interval.split('-'))) for interval in preferred if interval[0].isdigit()]
        not_preferred_intervals = [tuple(map(int, interval.split('-'))) for interval in not_preferred if interval[0].isdigit()]

        # Convert intervals from (8, 12) to [(8, 10), (10, 12)], etc.
        preferred_intervals_expanded = []
        not_preferred_intervals_expanded = []

        for interval in preferred_intervals:
            start = interval[0]
            end = interval[1]

            if start != end - 2:
                intervals = [str((i, i + 2)) for i in range(start, end, 2)]
                preferred_intervals_expanded.extend(intervals)
            else:
                preferred_intervals_expanded.append(str(interval))

        for interval in not_preferred_intervals:
            start = interval[0]
            end = interval[1]

            if start != end - 2:
                intervals = [str((i, i + 2)) for i in range(start, end, 2)]
                not_preferred_intervals_expanded.extend(intervals)
            else:
                not_preferred_intervals_expanded.append(str(interval))
            

        # Create the preferred and not preferred days.
        preferred_days = [day for day in preferred if not day[0].isdigit()]
        not_preferred_days = [day.strip('!') for day in not_preferred if not day[0].isdigit()]

        # Add the preferences to data.
        data[TEACHERS][teacher][PREFERRED] = preferred_days + preferred_intervals_expanded
        data[TEACHERS][teacher][NOT_PREFERRED] = not_preferred_days + not_preferred_intervals_expanded

        data[TEACHERS][teacher][PREFERRED_DAYS] = preferred_days
        data[TEACHERS][teacher][NOT_PREFERRED_DAYS] = not_preferred_days
        data[TEACHERS][teacher][PREFERRED_INTERVALS] = preferred_intervals_expanded
        data[TEACHERS][teacher][NOT_PREFERRED_INTERVALS] = not_preferred_intervals_expanded

    return data

# A* Algorithm

`State` data structure used by the A* implementation.

In [ ]:
class State:
    def __init__(self, cost, state):
        self.state = state
        self.cost = cost

    def __lt__(self, other):
        return self.cost < other.cost

`init_empty_timetable()` Returns an empty timetable.



In [ ]:
def init_empty_timetable(data):
    timetable = {day: {interval: {classroom: None for classroom in data[CLASSROOMS]} for interval in data[INTERVALS]} for day in data[DAYS]}
    return timetable

`is_covered()` Checks the timetable to see if the subject is fully covered.

In [ ]:
def is_covered(state, subject, data):
    student_count = 0

    # Sum the capacity of all assigned classrooms where the subject is taught. 
    for day in state:
        for interval in state[day]:
            for classroom in state[day][interval]:
                if state[day][interval][classroom] is not None and state[day][interval][classroom][1] == subject:
                    student_count += data[CLASSROOMS][classroom][CAPACITY]

    return student_count >= data[SUBJECTS][subject]

`check_hard_constraints()` Returns 1 if any hard constraint is violated and 0 otherwise.\
`check_soft_constraints()` Returns the number of soft constraints violated.\
`generate_next_states()` Generates the neighbors of the given state.

In [ ]:
def check_hard_constraints(state, data):
    teacher_counts = {teacher: 0 for teacher in data[TEACHERS]}

    for day in state:
        for interval in state[day]:
            for classroom in state[day][interval]:
                if state[day][interval][classroom] is None:
                    continue

                teacher = state[day][interval][classroom][0]
                subject = state[day][interval][classroom][1]

                # Check if a subject cannot be taught in the assigned classroom.
                if subject not in data[CLASSROOMS][classroom][SUBJECTS]:
                    return 1
                
                # Check if a teacher is assigned to more than MAX_INTERVALS classes.
                teacher_counts[teacher] += 1

                if teacher_counts[teacher] > MAX_INTERVALS:
                    return 1
                
            
            # Check if a teacher is assigned to more than one class at the same time.
            teachers = [state[day][interval][classroom][0] for classroom in state[day][interval] if state[day][interval][classroom] is not None]
            
            if len(teachers) != len(set(teachers)):
                return 1
            
    return 0
            

def check_soft_constraints(state, data):
    soft_cost = 0
    
    for day in state:
        for interval in state[day]:
            for classroom in state[day][interval]:
                if state[day][interval][classroom] is None:
                    continue

                teacher = state[day][interval][classroom][0]

                # Check if the teacher prefers the assigned interval.
                if interval in data[TEACHERS][teacher][NOT_PREFERRED]:
                    soft_cost += 1

                # Check if the teacher prefers the assigned day.
                if day in data[TEACHERS][teacher][NOT_PREFERRED]:
                    soft_cost += 1

    return soft_cost


def generate_next_states(state, subject_scores, data):
    next_states = []

    for subject in subject_scores:
        if is_covered(state, subject, data):
            continue

        # Search for a slot.
        for day in data[DAYS]:
            for interval in data[INTERVALS]:
                for classroom in data[CLASSROOMS]:

                    if state[day][interval][classroom] is None:

                        # Slot found, generate all combinations that also respect the hard constraints.
                        for teacher in data[TEACHERS]:
                            if subject in data[TEACHERS][teacher][SUBJECTS]:
                                new_state = deepcopy(state)
                                new_state[day][interval][classroom] = (teacher, subject)

                                if check_hard_constraints(new_state, data) == 0:
                                    next_states.append(new_state)
                    else:
                        continue

    return next_states

`count_assigned_students()` Returns the sum of the capacities of the assigned classrooms.\
`heuristic` Heuristic function used by the A* algorithm.

In [ ]:
def count_assigned_students(state, subject, data):
    student_count = 0

    for day in state:
        for interval in state[day]:
            for classroom in state[day][interval]:
                if state[day][interval][classroom] is None:
                    continue

                if state[day][interval][classroom][1] == subject:
                    student_count += data[CLASSROOMS][classroom][CAPACITY]

    return student_count


def heuristic(state, data):

    # Get the total number of students.
    total_students = sum(data[SUBJECTS].values())

    # Subtract the number of students assigned to a subject.
    for subject in data[SUBJECTS]:
        total_students -= count_assigned_students(state, subject, data)

    return check_soft_constraints(state, data) + total_students

`all_covered()` Checks if all subjects are covered.\
`is_final()` Checks if the given state is final.

In [ ]:
def all_covered(state, data):
    return all(is_covered(state, subject, data) for subject in data[SUBJECTS])

def is_final(state, acceptable_cost, data):
    return all_covered(state, data) and check_hard_constraints(state, data) == 0 and check_soft_constraints(state, data) <= acceptable_cost

`astar()` The A* algorithm.

In [ ]:
def astar(initial_state, acceptable_cost, data):

    subject_scores = {subject: 0 for subject in data[SUBJECTS]}

    for classroom in data[CLASSROOMS]:
        for subject in data[CLASSROOMS][classroom][SUBJECTS]:
            subject_scores[subject] += 1 / len(data[CLASSROOMS][classroom][SUBJECTS])

    sorted_subjects = list(subject_scores.keys())
    sorted_subjects.sort(key=lambda subject: subject_scores[subject])

    sorted_subjects = sorted(sorted_subjects, key=lambda subject: data[SUBJECTS][subject])

    frontier = []
    heappush(frontier, State(heuristic(initial_state, data), initial_state))

    while frontier:
        current_state = heappop(frontier).state

        if is_final(current_state, acceptable_cost, data):
            return current_state

        for next_state in generate_next_states(current_state, sorted_subjects, data):
            next_cost = heuristic(next_state, data)
            
            if check_soft_constraints(next_state, data) > acceptable_cost:
                continue

            heappush(frontier, State(next_cost, next_state))        
    
    return current_state

def run_astar(input_file):
    # Read input data.
    data = parse_input_file('inputs/' + input_file + '.yaml')

    # Initialize the empty timetable.
    initial_state = init_empty_timetable(data)

    # Run and time the A* algorithm.
    start = time.time()
    result = astar(initial_state, input_files[input_file], data)
    end = time.time()

    print(f'[A*] {input_file} Execution time: {round(end - start, 2)} seconds. \
        Hard constraints violated: {check_hard_constraints(result, data)}. Soft constraints violated: {check_soft_constraints(result, data)}.\n')
    
    return result

def transform_intervals(timetable):

    new_timetable = {}

    for day in timetable:
        new_timetable[day] = {}

        for interval in timetable[day]:
            temp = interval.strip('()').split(',')
            new_interval = (int(temp[0]), int(temp[1]))
            new_timetable[day][new_interval] = timetable[day][interval]

    return new_timetable  

def test_astar():
    for input_file in input_files:
        result = run_astar(input_file)
        transformed = transform_intervals(result)
        
        with open('outputs/' + input_file + '.txt', 'w') as f:
           f.write(utils.pretty_print_timetable(transformed, 'inputs/' + input_file + '.yaml'))

        print(utils.pretty_print_timetable(transformed, 'inputs/' + input_file + '.yaml'), end='\n')

test_astar()

`run_astar()` Runs A* on the specified input file. \
`transform_intervals()` Transforms the interval keys from string to tuple. \
`test_astar()` Runs the A* algorithm for all input files.

In [ ]:
def run_astar(input_file):
    # Read input data.
    data = parse_input_file('inputs/' + input_file + '.yaml')

    # Initialize the empty timetable.
    initial_state = init_empty_timetable(data)

    # Run and time the A* algorithm.
    start = time.time()
    result = astar(initial_state, input_files[input_file], data)
    end = time.time()

    print(f'[A*] {input_file} Execution time: {round(end - start, 2)} seconds. \
        Hard constraints violated: {check_hard_constraints(result, data)}. Soft constraints violated: {check_soft_constraints(result, data)}.\n')
    
    return result

def transform_intervals(timetable):

    new_timetable = {}

    for day in timetable:
        new_timetable[day] = {}

        for interval in timetable[day]:
            temp = interval.strip('()').split(',')
            new_interval = (int(temp[0]), int(temp[1]))
            new_timetable[day][new_interval] = timetable[day][interval]

    return new_timetable  

def test_astar():
    for input_file in input_files:
        result = run_astar(input_file)
        transformed = transform_intervals(result)
        
        with open('outputs/' + input_file + '.txt', 'w') as f:
           f.write(utils.pretty_print_timetable(transformed, 'inputs/' + input_file + '.yaml'))

        print(utils.pretty_print_timetable(transformed, 'inputs/' + input_file + '.yaml'), end='\n')

test_astar()

# PCSP Algorithm

`get_constraints()` Returns the constraints that implicate the variable. \
`check_constraint()` Checks if the solution satisfies the constraint. \
`fixed_constraint()` Returns only the constraints that can be evaluated in the partial solution.

In [ ]:
def get_constraints(var, constraints):
    return [constraint for constraint in constraints if var in constraint[0]]

def check_constraint(solution, constraint):
    return constraint[1](*(solution[var][0] for var in constraint[0]))
    
def fixed_constraints(solution, constraints):
    return [constraint for constraint in constraints if all(var in solution for var in constraint[0])]

In [ ]:
def is_covered2(solution, subject, data):
    student_count = 0

    # Sum the capacity of all assigned classrooms where the subject is taught. 
    for var in solution:
        if solution[var] != (None, None) and solution[var][1] == subject:
            student_count += data[CLASSROOMS][var[2]][CAPACITY]

    return student_count >= data[SUBJECTS][subject]

def check_all_covered(solution, data):
    return all(is_covered2(solution, subject, data) for subject in data[SUBJECTS])


def compute_new_domains(solution, domains, data):
    covered = {subject: is_covered2(solution, subject, data) for subject in data[SUBJECTS]}
    new_domains = {var: list(filter(lambda val: val[1] is None or not covered[val[1]], domains[var])) for var in domains}

    return new_domains

def check_hard_constraints3(solution, data):
    
    # Check if the same teacher is assigned to two classes at the same time.
    teacher_intervals = {}
    teacher_intervals_count = {teacher: 0 for teacher in data[TEACHERS]}

    for (day, interval, classroom), (teacher, subject) in solution.items():
        if teacher is None:
            continue
            
        teacher_intervals_count[teacher] += 1

        if teacher_intervals_count[teacher] > MAX_INTERVALS:
            return False

        if teacher not in teacher_intervals:
            teacher_intervals[teacher] = set()

        if (day, interval) in teacher_intervals[teacher]:
            return False
        else:
            teacher_intervals[teacher].add((day, interval))

    if not all(count <= MAX_INTERVALS for count in teacher_intervals_count.values()):
        return False
    
    return True
    

def pcsp(vars, domains, constraints, acceptable_cost, solution, cost, data, max_iters = 10000):
    global best_solution
    global best_cost
  
    domains = compute_new_domains(solution, domains, data)

    if not vars:
        best_solution = deepcopy(solution)
        best_cost = cost

        if best_cost <= acceptable_cost and check_all_covered(solution, data) and check_hard_constraints3(solution, data):
            return True
        
    elif not domains[vars[0]]:
        return False
    elif cost == best_cost:
        return False
    else:
        
        var = vars[0]
        val = domains[var].pop(0)

        new_solution = deepcopy(solution)
        new_solution[var] = val

        if not check_hard_constraints3(new_solution, data):
            return False

        eval_soft_constraints = fixed_constraints(new_solution, constraints)
        new_cost = 0
                
        for constraint in eval_soft_constraints:
            if not check_constraint(new_solution, constraint):
                new_cost += 1
                
        if new_cost < best_cost and new_cost <= acceptable_cost:
            if pcsp(vars[1:], deepcopy(domains), constraints, acceptable_cost, new_solution, new_cost, data) and check_all_covered(new_solution, data):
                return True
            
        return pcsp(vars, deepcopy(domains), constraints, acceptable_cost, solution, cost, data)



`transform_solution()` Transforms the solution to dictionary: day -> interval -> classroom -> (teacher, subject) \
`run_pcsp()` Runs the PCSP algorithm on the specified input.
`test_pcsp()` Runs the PCSP algorithm for all input files.

In [ ]:
def transform_solution(solution, data):
    new_solution = init_empty_timetable(data)

    for var in solution:
        day = var[0]
        interval = var[1]
        classroom = var[2]

        teacher = solution[var][0]
        subject = solution[var][1]

        if teacher is not None and subject is not None:
            new_solution[day][interval][classroom] = (teacher, subject)
        else:
            new_solution[day][interval][classroom] = None

    return new_solution


def run_pcsp(input_file):
    global best_solution
    global best_cost
    
    data = parse_input_file('inputs/' + input_file + '.yaml')

    # Set up variables as product of days, intervals and classrooms.
    vars = [(day, interval, classroom) for day in data[DAYS] for interval in data[INTERVALS] for classroom in data[CLASSROOMS]]

    # Set up domains as tuples of teacher and subjects.
    domains = {var: [(teacher, subject)
                    for teacher in data[TEACHERS]
                    for subject in data[TEACHERS][teacher][SUBJECTS] if subject in data[CLASSROOMS][var[2]][SUBJECTS]]
                    for var in vars}


    # Sort the domains by the number of students assigned to the subject, ascending.
    for var in domains:
        domains[var] = sorted(domains[var], key=lambda x: data[SUBJECTS][x[1]])
        domains[var].append((None, None))      

    # Add soft constraints.
    soft_constraints = [([var], lambda teacher, var=var: teacher is None or var[0] in data[TEACHERS][teacher][PREFERRED_DAYS]) for var in vars]
    soft_constraints += [([var], lambda teacher, var=var: teacher is None or var[1] in data[TEACHERS][teacher][PREFERRED_INTERVALS]) for var in vars]

    best_solution = {}
    best_cost = len(soft_constraints)

    # Run and time the PCSP algorithm.
    start = time.time()
    pcsp(vars, domains, soft_constraints, input_files[input_file], {}, input_files[input_file], data)
    end = time.time()
    
    print(f'[PCSP] {input_file} Execution time: {round(end - start, 2)} seconds. \
        Hard constraints violated: {check_hard_constraints3(best_solution, data)}. Soft constraints violated: {best_cost}.\n')
    
    return transform_solution(best_solution, data)

def test_pcsp():
    for input_file in input_files.keys():
        result = run_pcsp(input_file)
        transformed = transform_intervals(result)
        
        with open('outputs/' + input_file + '.txt', 'w') as f:
            f.write(utils.pretty_print_timetable(transformed, 'inputs/' + input_file + '.yaml'))

test_pcsp()